In [28]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import r2_score

In [29]:
df = pd.read_csv('../input/nba-games/ranking.csv')
df.dropna(inplace=True)
x = df[['TEAM_ID', 'G', 'W']]
y = df[['W_PCT']]

In [30]:
scaler_x = StandardScaler()
scaler_y = StandardScaler()
x = scaler_x.fit_transform(x)
y = scaler_y.fit_transform(y)

In [31]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=0)

In [32]:
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
seq_len = X_train[0].shape[0]

In [33]:
class NBA(Dataset):
    def __init__(self, X_train, y_train):
        super(Dataset, self).__init__()
        self.X_train = X_train
        self.y_train = y_train

    def __len__(self):
        return len(self.X_train)
    
    def __getitem__(self, idx):
        x = self.X_train[idx]
        y = self.y_train[idx]
        return x, y

In [34]:
hidden_size = 512
num_layers = 2
learning_rate = 0.001
batch_size = 64
epoch_size = 10

In [35]:
train_dataset = NBA(X_train, y_train)
test_dataset = NBA(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [36]:
class RNN(nn.Module):
    def __init__(self, input_feature_size, hidden_size, num_layers):
        super(RNN, self).__init__()
        self.lstm = nn.LSTM(input_feature_size, hidden_size, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [37]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
rnn = RNN(3, hidden_size, num_layers).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=0.0001)

In [38]:
rnn.train()
for epoch in range(epoch_size):
    loss = 0.0

    for batch_idx, data in enumerate(train_loader):
        inputs, targets = data
        inputs.to(device)
        targets.to(device)

        optimizer.zero_grad()

        outputs = rnn(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        loss += loss.item()
        if batch_idx % 100 == 99:
            print(f'[{epoch + 1}, {batch_idx + 1:5d}] loss: {loss / 100:.3f}')
            loss = 0.0

In [39]:
print('Finished Training')

Finished Training


In [40]:
prediction = []
ground_truth = []

rnn.eval()
with torch.no_grad():
    for data in test_loader:
        inputs, targets = data
        inputs = inputs.to(device)

        ground_truth += targets.flatten().tolist()
        out = rnn(inputs).detach().cpu().flatten().tolist()
        prediction += out

In [ ]:
prediction = scaler_y.inverse_transform(prediction)
ground_truth = scaler_y.inverse_transform(ground_truth)

In [25]:
r2score = r2_score(prediction, ground_truth)

In [27]:
for i in range(len(prediction)):
    print("Prediction: ", prediction[i], ", ground truth: ", ground_truth[i])
    print(prediction[i] - ground_truth[i])

Prediction:  -0.2559939920902252 , ground truth:  -0.7979643749453682
0.541970382855143
Prediction:  0.3940427303314209 , ground truth:  1.1428929196988873
-0.7488501893674664
Prediction:  0.7869055867195129 , ground truth:  0.8084682781601846
-0.02156269144067169
Prediction:  -0.8791944980621338 , ground truth:  -1.0846140676928273
0.20541956963069352
Prediction:  -1.1786446571350098 , ground truth:  -1.1264171478851652
-0.05222750924984454
Prediction:  -0.093304842710495 , ground truth:  -0.06939640587890937
-0.023908436831585625
Prediction:  1.706756830215454 , ground truth:  1.5250925100288326
0.1816643201866215
Prediction:  -1.1488735675811768 , ground truth:  -1.0607265932972056
-0.08814697428397111
Prediction:  1.0343362092971802 , ground truth:  1.0533148907153063
-0.018978681418126087
Prediction:  0.8501476645469666 , ground truth:  0.8084682781601846
0.041679386386781925
Prediction:  1.7633066177368164 , ground truth:  1.7102204365949003
0.0530861811419161
Prediction:  0.6574